<a href="https://colab.research.google.com/github/rumman-adnan/Assignment-Sensors-Data/blob/main/FCN_FCN_PLUS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
from sklearn.preprocessing import StandardScaler
# Load the training and test data
train_path = '/content/drive/MyDrive/Python work/sim_data/simu_20000_0.1_90_140_train.npy'
test_path = '/content/drive/MyDrive/Python work/sim_data/simu_10000_0.1_141_178_test.npy'
train_data = np.load(train_path)
test_data = np.load(test_path)

# Extract the sensor data and the target variables (S and D)
X_train = train_data[:, :1000]
y_train = train_data[:, -2:]
X_test = test_data[:, :1000]
y_test = test_data[:, -2:]

In [3]:
X_train_mean = np.mean(X_train, axis = 1)
X_train_mean = np.reshape(X_train_mean, (-1,1))
X_train_std = np.std(X_train, axis = 1)
X_train_std = np.reshape(X_train_std, (-1,1))
X_train = (X_train - X_train_mean)/X_train_std


X_test_mean = np.mean(X_test, axis = 1)
X_test_mean = np.reshape(X_test_mean, (-1,1))
X_test_std = np.std(X_test, axis = 1)
X_test_std = np.reshape(X_test_std, (-1,1))
X_test = (X_test - X_test_mean)/X_test_std


In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.metrics import MeanAbsoluteError
from sklearn.model_selection import train_test_split

# # Load the complete training and testing data
# train_data = np.load('simu_20000_0.1_90_140_train.npy')
# test_data = np.load('simu_10000_0.1_141_178_test.npy')

# Separate features and labels
X_train_full = train_data[:, :1000]
y_train_full = train_data[:, -2]  # Only Systolic Blood Pressure
X_test = test_data[:, :1000]
y_test = test_data[:, -2]  # Only Systolic Blood Pressure

# Split training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.3, random_state=42, shuffle=True)

# Scale the features
X_train_mean = np.mean(X_train, axis=1)
X_train_mean = np.reshape(X_train_mean, (-1, 1))
X_train_std = np.std(X_train, axis=1)
X_train_std = np.reshape(X_train_std, (-1, 1))
X_train = (X_train - X_train_mean) / X_train_std

X_val_mean = np.mean(X_val, axis=1)
X_val_mean = np.reshape(X_val_mean, (-1, 1))
X_val_std = np.std(X_val, axis=1)
X_val_std = np.reshape(X_val_std, (-1, 1))
X_val = (X_val - X_val_mean) / X_val_std

X_test_mean = np.mean(X_test, axis=1)
X_test_mean = np.reshape(X_test_mean, (-1, 1))
X_test_std = np.std(X_test, axis=1)
X_test_std = np.reshape(X_test_std, (-1, 1))
X_test = (X_test - X_test_mean) / X_test_std

# Reshape data for FCN_PLUS
X_train = X_train[:, :, np.newaxis]
X_val = X_val[:, :, np.newaxis]
X_test = X_test[:, :, np.newaxis]

# Build the FCN_PLUS model
model = models.Sequential([
    layers.Input(shape=(X_train.shape[1], 1)),
    layers.Conv1D(filters=128, kernel_size=8, padding='same'),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Conv1D(filters=256, kernel_size=5, padding='same'),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Conv1D(filters=128, kernel_size=3, padding='same'),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.GlobalAveragePooling1D(),
    layers.Dense(1)  # Single output node for Systolic Blood Pressure
])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=[MeanAbsoluteError()])

# Train the model for 70 epochs and capture loss after each epoch
for epoch in range(50):
    history = model.fit(X_train, y_train, epochs=1, batch_size=32, validation_data=(X_val, y_val), shuffle=False, verbose=1)

    train_loss = history.history['loss'][0]
    train_mae = history.history['mean_absolute_error'][0]

    val_loss = history.history['val_loss'][0]
    val_mae = history.history['val_mean_absolute_error'][0]

    # Evaluate the model on the test set
    test_loss, test_mae = model.evaluate(X_test, y_test, verbose=0)

    print(f"Epoch {epoch+1}, Train Loss: {train_loss}, Train MAE: {train_mae}, Val Loss: {val_loss}, Val MAE: {val_mae}, Test Loss: {test_loss}, Test MAE: {test_mae}")
    prediction = model.predict(X_test)
    mae_systolic = MeanAbsoluteError()
    mae_systolic.update_state(y_test, prediction)
    print("Mean absolute error for systolic is: ",mae_systolic.result().numpy())


438/438 [==============================] - 20s 14ms/step - loss: 8768.6826 - mean_absolute_error: 91.0489 - val_loss: 3204.9282 - val_mean_absolute_error: 54.8738
Epoch 1, Train Loss: 8768.6826171875, Train MAE: 91.04888916015625, Val Loss: 3204.92822265625, Val MAE: 54.873802185058594, Test Loss: 9582.33203125, Test MAE: 97.27973175048828
313/313 [==============================] - 1s 3ms/step
Mean absolute error for systolic is:  97.27976
438/438 [==============================] - 6s 13ms/step - loss: 991.3674 - mean_absolute_error: 25.4867 - val_loss: 2494.3137 - val_mean_absolute_error: 47.2188
Epoch 2, Train Loss: 991.367431640625, Train MAE: 25.486677169799805, Val Loss: 2494.313720703125, Val MAE: 47.21881103515625, Test Loss: 298.0421142578125, Test MAE: 14.16926097869873
313/313 [==============================] - 1s 3ms/step
Mean absolute error for systolic is:  14.169259
438/438 [==============================] - 6s 13ms/step - loss: 84.4829 - mean_absolute_error: 7.5677 - val

KeyboardInterrupt: ignored

In [5]:
y_pred_s = model.predict(X_test)


313/313 [==============================] - 1s 3ms/step


In [6]:
from tensorflow.keras.metrics import mean_absolute_error

In [7]:
def calc_mae(gt, pred):
    return mean_absolute_error(gt,pred)

# list1: label; list2: prediction
def plot_2vectors(label, pred, name):
    list1 = label
    list2 = pred
    if len(list2.shape) == 2:
        mae = calc_mae(list1[:,0], list2[:,0])
    else:
        mae = calc_mae(list1, list2)

    # zipped_lists = zip(list1, list2)
    # sorted_pairs = sorted(zipped_lists)

    # tuples = zip(*sorted_pairs)
    # list1, list2 = np.array([ list(tuple) for tuple in  tuples])

    # print(list1.shape)
    # print(list2.shape)

    sorted_id = sorted(range(len(list1)), key=lambda k: list1[k])

    plt.clf()
    plt.text(0,np.min(list2),f'MAE={mae}')

    # plt.plot(range(num_rows), list2, label=name + ' prediction')
    plt.scatter(np.arange(list2.shape[0]),list2[sorted_id],s = 1, alpha=0.5,label=f'{name} prediction', color='blue')

    plt.scatter(np.arange(list1.shape[0]),list1[sorted_id],s = 1, alpha=0.5,label=f'{name} label', color='red')

    # plt.plot(range(num_rows), list1, 'r.', label=name + ' label')
    plt.title('Prediction Results')
    plt.xlabel('Number of Sample')
    plt.ylabel('Value')
    plt.legend()
    plt.savefig(f'{name}.png')
    print(f'Saved plot to {name}.png')
    plt.show()

In [17]:
model.save('model.h5')

In [18]:
systolic_model = model
model.save('systolic_model.h5')

In [9]:
plot_2vectors(y_test,y_pred_s,'plot1')

IndexError: ignored

In [10]:
y1=y_pred_s
print(y1)

[[183.70036]
 [177.63573]
 [155.28947]
 ...
 [163.15134]
 [163.63686]
 [161.78532]]


In [15]:
y1.shape[1]

1

In [16]:
y_test.shape[1]

IndexError: ignored

In [23]:
# prompt: scatter plot between y_test and y_pred_s

plot_2vectors(y_test[1],y_pred_s,'plot1')


IndexError: ignored